In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("golammostofas/the-verdict")

print("Path to dataset files:", path)

100%|██████████| 165k/165k [00:00<00:00, 1.11MB/s]

Extracting files...


Path to dataset files: C:\Users\pirgh\.cache\kagglehub\datasets\golammostofas\the-verdict\versions\1


In [3]:
import shutil
import kagglehub
import os

project_path = r"D:\NLP_Projects\fcc_ai_engineering_aws"
path = kagglehub.dataset_download("golammostofas/the-verdict")
target_folder = os.path.join(project_path, "the-verdict")

shutil.copytree(path, target_folder, dirs_exist_ok=True)

'D:\\NLP_Projects\\fcc_ai_engineering_aws\\the-verdict'

## Tokenization

In [4]:
with open('the-verdict.txt', 'r', encoding='utf-8') as f:
    raw_text = f.read()
    
print('Total number of character": ', len(raw_text))
print(raw_text[:99])  # Print the first 500 characters

Total number of character":  20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [6]:
import re

text = "Hello world. This, is a text."
result = re.split(r'(\s)', text)

print(result)  # ['Hello', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'text.']

['Hello', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'text.']


In [11]:
# preprocessed = re.split(r'([,.?_!"()\']--|\s)', raw_text)
preprocessed = re.findall(r"[A-Za-z]+(?:'[A-Za-z]+)?|[0-9]+|[^\w\s]", raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]

print(preprocessed[:30])  # Print the first 20 tokens

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '-', '-', 'though', 'a', 'good', 'fellow', 'enough', '-', '-', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that']


In [12]:
 print(len(preprocessed))

4669


In [14]:
# finding the unique tokens
unique_tokens = sorted(set(preprocessed))
print(len(unique_tokens))


1155


In [15]:
vocab = {token:integer for integer, token in enumerate(unique_tokens)}

In [16]:
for i, item in enumerate(vocab.items()):
    if i < 50:
        print(item)

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('-', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
("Don't", 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
("Gisburn's", 39)
('Gisburns', 40)
('Grafton', 41)
('Greek', 42)
('Grindle', 43)
("Grindle's", 44)
('Grindles', 45)
('HAD', 46)
('Had', 47)
('Hang', 48)
('Has', 49)


In [ ]:
s